In [13]:
# 11/18/2021-1
from __future__ import absolute_import, division
from IPython.display import clear_output
import glob, os
from os.path import join
import tensorflow as tf
from tensorflow import keras

contentPath = os.getcwd()
dataPath = join(contentPath, 'data')
testPath = join(contentPath, 'TestImages')

In [2]:
from tensorflow import keras
from tensorflow.keras import layers

data_augmentation = keras.Sequential(
    [layers.RandomFlip("horizontal"), layers.RandomRotation(0.1),]
)

2021-11-25 15:20:00.882228: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2021-11-25 15:20:00.882292: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2021-11-25 15:20:00.882324: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (jupyter-iypc-2dteam-2dcv2-5fimages-2dgeh7bvzd): /proc/driver/nvidia/version does not exist
2021-11-25 15:20:00.882763: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
base_model = keras.applications.MobileNetV3Small(
    weights="imagenet",  # Load weights pre-trained on ImageNet.
    input_shape=(224, 224, 3),
    include_top=False,
)  # Do not include the ImageNet classifier at the top.

# Freeze the base_model
base_model.trainable = False

# Create new model on top
inputs = keras.Input(shape=(224, 224, 3))
x = data_augmentation(inputs)  # Apply random data augmentation

# Pre-trained Xception weights requires that input be scaled
# from (0, 255) to a range of (-1., +1.), the rescaling layer
# outputs: `(inputs * scale) + offset`
scale_layer = keras.layers.Rescaling(scale=1 / 127.5, offset=-1)
x = scale_layer(x)

# The base model contains batchnorm layers. We want to keep them in inference mode
# when we unfreeze the base model for fine-tuning, so we make sure that the
# base_model is running in inference mode here.
x = base_model(x, training=False)
x = keras.layers.GlobalAveragePooling2D()(x)
x = keras.layers.Dropout(0.2)(x)  # Regularize with dropout
outputs = keras.layers.Dense(1)(x)
model = keras.Model(inputs, outputs)

model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 sequential (Sequential)     (None, 224, 224, 3)       0         
                                                                 
 rescaling_1 (Rescaling)     (None, 224, 224, 3)       0         
                                                                 
 MobilenetV3small (Functiona  (None, 1, 1, 1024)       1529968   
 l)                                                              
                                                                 
 global_average_pooling2d_1   (None, 1024)             0         
 (GlobalAveragePooling2D)                                        
                                                                 
 dropout (Dropout)           (None, 1024)              0     

In [4]:
cv2Path=join(contentPath, 'CV2Images')

from tensorflow.keras.utils import image_dataset_from_directory
train_ds = image_dataset_from_directory(
    cv2Path,
    color_mode='rgb',
    validation_split=0.2,
    subset="training",
    seed=456,
    image_size=(224, 224),
    batch_size=4)
print()
validation_ds = image_dataset_from_directory(
    cv2Path,
    color_mode='rgb',
    validation_split=0.2,
    subset="validation",
    seed=456,
    image_size=(224, 224),
    batch_size=4)
print()
# clear_output()

Found 216 files belonging to 2 classes.
Using 173 files for training.

Found 216 files belonging to 2 classes.
Using 43 files for validation.



In [5]:
model.compile(
    optimizer=keras.optimizers.Adam(),
    loss=keras.losses.BinaryCrossentropy(from_logits=True),
    metrics=[keras.metrics.BinaryAccuracy()],
)

epochs = 3
model.fit(train_ds, epochs=epochs, validation_data=validation_ds)
# clear_output()

Epoch 1/3
44/44 [==============================] - 15s 242ms/step - loss: 0.3819 - binary_accuracy: 0.8382 - val_loss: 0.3168 - val_binary_accuracy: 0.9070
Epoch 2/3
44/44 [==============================] - 9s 207ms/step - loss: 0.3799 - binary_accuracy: 0.8844 - val_loss: 0.3087 - val_binary_accuracy: 0.9070
Epoch 3/3
44/44 [==============================] - 9s 209ms/step - loss: 0.3875 - binary_accuracy: 0.8844 - val_loss: 0.3107 - val_binary_accuracy: 0.9070


In [6]:
# Unfreeze the base_model. Note that it keeps running in inference mode
# since we passed `training=False` when calling it. This means that
# the batchnorm layers will not update their batch statistics.
# This prevents the batchnorm layers from undoing all the training
# we've done so far.
base_model.trainable = True
# model.summary()

model.compile(
    optimizer=keras.optimizers.Adam(1e-5),  # Low learning rate
    loss=keras.losses.BinaryCrossentropy(from_logits=True),
    metrics=[keras.metrics.BinaryAccuracy()],
)

epochs = 1
history=model.fit(train_ds,
                  epochs=epochs,
                  validation_data=validation_ds)

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 sequential (Sequential)     (None, 224, 224, 3)       0         
                                                                 
 rescaling_1 (Rescaling)     (None, 224, 224, 3)       0         
                                                                 
 MobilenetV3small (Functiona  (None, 1, 1, 1024)       1529968   
 l)                                                              
                                                                 
 global_average_pooling2d_1   (None, 1024)             0         
 (GlobalAveragePooling2D)                                        
                                                                 
 dropout (Dropout)           (None, 1024)              0     

In [7]:
modelName = base_model.name + '_retrained.h5'
savePath = join(contentPath, modelName)
print(savePath)
model.save(savePath)

/home/jovyan/MobilenetV3small_retrained.h5


/srv/conda/envs/notebook/lib/python3.7/site-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)


In [8]:
new_model = tf.keras.models.load_model(savePath)
# new_model.summary() # Check its architecture
inputShape=new_model.input_shape
new_model.build(input_shape=inputShape)
help(new_model.predict)

Help on method predict in module keras.engine.training:

predict(x, batch_size=None, verbose=0, steps=None, callbacks=None, max_queue_size=10, workers=1, use_multiprocessing=False) method of keras.engine.functional.Functional instance
    Generates output predictions for the input samples.
    
    Computation is done in batches. This method is designed for performance in
    large scale inputs. For small amount of inputs that fit in one batch,
    directly using `__call__()` is recommended for faster execution, e.g.,
    `model(x)`, or `model(x, training=False)` if you have layers such as
    `tf.keras.layers.BatchNormalization` that behaves differently during
    inference. Also, note the fact that test loss is not affected by
    regularization layers like noise and dropout.
    
    Args:
        x: Input samples. It could be:
          - A Numpy array (or array-like), or a list of arrays
            (in case the model has multiple inputs).
          - A TensorFlow tensor, or a lis

In [14]:
os.chdir(testPath)
testGlobList=[]
testGlobList=glob.glob('**')

FileNotFoundError: [Errno 2] No such file or directory: '/home/jovyan/TestImages/TestImages'

In [ ]:
number_of_steps=224 * 224
result=new_model.predict(img,
                        batch_size=1,
                        verbose=1,
                        steps=number_of_steps,
                        max_queue_size=2)